In [15]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import StandardScaler
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import ta
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier


file_path = 'EA_stock_data.csv'
# Assuming ea_df is your DataFrame loaded with stock data including 'Open', 'High', 'Low', 'Close', 'Volume'
ea_df = pd.read_csv(file_path)
# Calculate Moving Average
ea_df['SMA_10'] = ea_df['Close'].rolling(window=5).mean()

# Calculate MACD
ea_df['MACD'] = ta.trend.MACD(ea_df['Close']).macd()

# Calculate ADX
ea_df['ADX'] = ta.trend.ADXIndicator(ea_df['High'], ea_df['Low'], ea_df['Close'], window=20).adx()
ea_df['Smoothed_Close'] = ea_df['Close'].ewm(alpha=0.1).mean()
# Drop any rows with NaN values that were introduced by the indicator calculations
ea_df.dropna(inplace=True)

# Define the target variable: 1 if the price goes up next week, 0 otherwise
ea_df['Target'] = (ea_df['Adj Close'].shift(-30) > ea_df['Adj Close']).astype(int)
ea_df['Target'] = ea_df['Target'].astype('category')

# Update features list to include the new technical indicators
features = ['Open', 'Low', 'Close', 'Volume','SMA_10', 'MACD','ADX']
X = ea_df[features]

# Standardize the features

y = ea_df['Target'].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# Initialize and train the SVM model
rf_classifier = RandomForestClassifier(n_estimators=476, random_state=42,max_depth= 20) 
rf_classifier.fit(X_train, y_train)

# Make predictions and evaluate the model
predictions = rf_classifier.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions, zero_division=0))

Accuracy: 0.7125506072874493
              precision    recall  f1-score   support

           0       0.62      0.75      0.68       100
           1       0.80      0.69      0.74       147

    accuracy                           0.71       247
   macro avg       0.71      0.72      0.71       247
weighted avg       0.73      0.71      0.72       247

